<a href="https://colab.research.google.com/github/FaribaAzizian/BigData-1402/blob/main/BigData_HW8_FaribaAzizian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install java, it is not needed if installed
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download and install cassandra
!wget -q https://archive.apache.org/dist/cassandra/4.0.9/apache-cassandra-4.0.9-bin.tar.gz
# unzip the cassandra file to the current folder
!tar -xzvf apache-cassandra-4.0.9-bin.tar.gz

apache-cassandra-4.0.9/bin/
apache-cassandra-4.0.9/conf/
apache-cassandra-4.0.9/conf/triggers/
apache-cassandra-4.0.9/doc/
apache-cassandra-4.0.9/doc/cql3/
apache-cassandra-4.0.9/lib/
apache-cassandra-4.0.9/lib/sigar-bin/
apache-cassandra-4.0.9/pylib/
apache-cassandra-4.0.9/pylib/cqlshlib/
apache-cassandra-4.0.9/pylib/cqlshlib/test/
apache-cassandra-4.0.9/pylib/cqlshlib/test/config/
apache-cassandra-4.0.9/tools/
apache-cassandra-4.0.9/tools/bin/
apache-cassandra-4.0.9/tools/lib/
apache-cassandra-4.0.9/CASSANDRA-14092.txt
apache-cassandra-4.0.9/CHANGES.txt
apache-cassandra-4.0.9/LICENSE.txt
apache-cassandra-4.0.9/NEWS.txt
apache-cassandra-4.0.9/NOTICE.txt
apache-cassandra-4.0.9/conf/README.txt
apache-cassandra-4.0.9/conf/cassandra-env.sh
apache-cassandra-4.0.9/conf/cassandra-jaas.config
apache-cassandra-4.0.9/conf/cassandra-rackdc.properties
apache-cassandra-4.0.9/conf/cassandra-topology.properties
apache-cassandra-4.0.9/conf/cassandra.yaml
apache-cassandra-4.0.9/conf/commitlog_archivin

In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [3]:
# set spark path to system path environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["CASSANDRA_HOME"] = "/content/apache-cassandra-4.0.9"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [4]:
!./apache-cassandra-4.0.9/bin/cassandra -R
!echo "Waiting for 10 secs until cassandra services are up and running"
!sleep 10

CompilerOracle: dontinline org/apache/cassandra/db/Columns$Serializer.deserializeLargeSubset (Lorg/apache/cassandra/io/util/DataInputPlus;Lorg/apache/cassandra/db/Columns;I)Lorg/apache/cassandra/db/Columns;
CompilerOracle: dontinline org/apache/cassandra/db/Columns$Serializer.serializeLargeSubset (Ljava/util/Collection;ILorg/apache/cassandra/db/Columns;ILorg/apache/cassandra/io/util/DataOutputPlus;)V
CompilerOracle: dontinline org/apache/cassandra/db/Columns$Serializer.serializeLargeSubsetSize (Ljava/util/Collection;ILorg/apache/cassandra/db/Columns;I)I
CompilerOracle: dontinline org/apache/cassandra/db/commitlog/AbstractCommitLogSegmentManager.advanceAllocatingFrom (Lorg/apache/cassandra/db/commitlog/CommitLogSegment;)V
CompilerOracle: dontinline org/apache/cassandra/db/transform/BaseIterator.tryGetMoreContents ()Z
CompilerOracle: dontinline org/apache/cassandra/db/transform/StoppingTransformation.stop ()V
CompilerOracle: dontinline org/apache/cassandra/db/transform/StoppingTransforma

In [5]:
!pip install cassandra-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 36.3 MB/s eta 0:00:00


In [6]:
!pip install -q findspark

In [7]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.3.2-bin-hadoop3'

In [8]:
!pip install pyspark --quiet
!pip install -q handyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [150]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import col,isnan, when, count

spark = SparkSession \
    .builder \
    .appName("Uber_dataset.") \
    .getOrCreate()

uber_data = spark.read.options(inferSchema='True',header='True',sep=',').csv("/content/drive/MyDrive/uber-raw-data-aug14.csv")
for col in uber_data.columns:
  print(col, "\t", "with null values: ", uber_data.filter(uber_data[col].isNull()).count())
uber_data.show()

Date/Time 	 with null values:  0
Lat 	 with null values:  0
Lon 	 with null values:  0
Base 	 with null values:  0
+----------------+-------+--------+------+
|       Date/Time|    Lat|     Lon|  Base|
+----------------+-------+--------+------+
|8/1/2014 0:03:00|40.7366|-73.9906|B02512|
|8/1/2014 0:09:00| 40.726|-73.9918|B02512|
|8/1/2014 0:12:00|40.7209|-74.0507|B02512|
|8/1/2014 0:12:00|40.7387|-73.9856|B02512|
|8/1/2014 0:12:00|40.7323|-74.0077|B02512|
|8/1/2014 0:13:00|40.7349|-74.0033|B02512|
|8/1/2014 0:15:00|40.7279|-73.9542|B02512|
|8/1/2014 0:17:00| 40.721|-73.9937|B02512|
|8/1/2014 0:19:00|40.7195| -74.006|B02512|
|8/1/2014 0:20:00|40.7448|-73.9799|B02512|
|8/1/2014 0:21:00|40.7399|-74.0057|B02512|
|8/1/2014 0:25:00|40.7651|-73.9683|B02512|
|8/1/2014 0:27:00|40.7354|-74.0081|B02512|
|8/1/2014 0:29:00|40.7339|-74.0028|B02512|
|8/1/2014 0:29:00|40.7364|-74.0301|B02512|
|8/1/2014 0:29:00|40.7364|-74.0301|B02512|
|8/1/2014 0:30:00|40.7252|-73.9516|B02512|
|8/1/2014 0:30:00|40.7433

In [ ]:
uber_data.printSchema()

root
 |-- DateTimeName: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Lon: double (nullable = true)
 |-- Base: string (nullable = true)



In [11]:
#for rename columnName
uber_data = uber_data.withColumnRenamed("Date/Time","DateTimeName")
uber_data.show()

+----------------+-------+--------+------+
|    DateTimeName|    Lat|     Lon|  Base|
+----------------+-------+--------+------+
|8/1/2014 0:03:00|40.7366|-73.9906|B02512|
|8/1/2014 0:09:00| 40.726|-73.9918|B02512|
|8/1/2014 0:12:00|40.7209|-74.0507|B02512|
|8/1/2014 0:12:00|40.7387|-73.9856|B02512|
|8/1/2014 0:12:00|40.7323|-74.0077|B02512|
|8/1/2014 0:13:00|40.7349|-74.0033|B02512|
|8/1/2014 0:15:00|40.7279|-73.9542|B02512|
|8/1/2014 0:17:00| 40.721|-73.9937|B02512|
|8/1/2014 0:19:00|40.7195| -74.006|B02512|
|8/1/2014 0:20:00|40.7448|-73.9799|B02512|
|8/1/2014 0:21:00|40.7399|-74.0057|B02512|
|8/1/2014 0:25:00|40.7651|-73.9683|B02512|
|8/1/2014 0:27:00|40.7354|-74.0081|B02512|
|8/1/2014 0:29:00|40.7339|-74.0028|B02512|
|8/1/2014 0:29:00|40.7364|-74.0301|B02512|
|8/1/2014 0:29:00|40.7364|-74.0301|B02512|
|8/1/2014 0:30:00|40.7252|-73.9516|B02512|
|8/1/2014 0:30:00|40.7433| -73.986|B02512|
|8/1/2014 0:34:00|40.7437|-73.9884|B02512|
|8/1/2014 0:36:00|40.7406|-74.0077|B02512|
+----------

In [ ]:
uber_data.printSchema()

root
 |-- DateTimeName: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Lon: double (nullable = true)
 |-- Base: string (nullable = true)



In [12]:
#for change string type to timestamp

uber_data = uber_data.withColumn("DateTimeName", to_timestamp("DateTimeName",'M/d/yyyy H:mm:ss'))
uber_data.show()

+-------------------+-------+--------+------+
|       DateTimeName|    Lat|     Lon|  Base|
+-------------------+-------+--------+------+
|2014-08-01 00:03:00|40.7366|-73.9906|B02512|
|2014-08-01 00:09:00| 40.726|-73.9918|B02512|
|2014-08-01 00:12:00|40.7209|-74.0507|B02512|
|2014-08-01 00:12:00|40.7387|-73.9856|B02512|
|2014-08-01 00:12:00|40.7323|-74.0077|B02512|
|2014-08-01 00:13:00|40.7349|-74.0033|B02512|
|2014-08-01 00:15:00|40.7279|-73.9542|B02512|
|2014-08-01 00:17:00| 40.721|-73.9937|B02512|
|2014-08-01 00:19:00|40.7195| -74.006|B02512|
|2014-08-01 00:20:00|40.7448|-73.9799|B02512|
|2014-08-01 00:21:00|40.7399|-74.0057|B02512|
|2014-08-01 00:25:00|40.7651|-73.9683|B02512|
|2014-08-01 00:27:00|40.7354|-74.0081|B02512|
|2014-08-01 00:29:00|40.7339|-74.0028|B02512|
|2014-08-01 00:29:00|40.7364|-74.0301|B02512|
|2014-08-01 00:29:00|40.7364|-74.0301|B02512|
|2014-08-01 00:30:00|40.7252|-73.9516|B02512|
|2014-08-01 00:30:00|40.7433| -73.986|B02512|
|2014-08-01 00:34:00|40.7437|-73.9

In [13]:
### Extract day of month and day of week from date in pyspark
 
from pyspark.sql.functions import dayofmonth
from pyspark.sql.functions import to_date
from pyspark.sql.types import *
import pyspark.sql.functions as F
 
uber_data = uber_data.withColumn('dayofmonth',dayofmonth(uber_data.DateTimeName))
uber_data = uber_data.withColumn('dayofweek',dayofweek(uber_data.DateTimeName))
uber_data=uber_data.withColumn("DateTimeAsInteger", F.unix_timestamp(uber_data['DateTimeName']))

uber_data.show()

+-------------------+-------+--------+------+----------+---------+-----------------+
|       DateTimeName|    Lat|     Lon|  Base|dayofmonth|dayofweek|DateTimeAsInteger|
+-------------------+-------+--------+------+----------+---------+-----------------+
|2014-08-01 00:03:00|40.7366|-73.9906|B02512|         1|        6|       1406851380|
|2014-08-01 00:09:00| 40.726|-73.9918|B02512|         1|        6|       1406851740|
|2014-08-01 00:12:00|40.7209|-74.0507|B02512|         1|        6|       1406851920|
|2014-08-01 00:12:00|40.7387|-73.9856|B02512|         1|        6|       1406851920|
|2014-08-01 00:12:00|40.7323|-74.0077|B02512|         1|        6|       1406851920|
|2014-08-01 00:13:00|40.7349|-74.0033|B02512|         1|        6|       1406851980|
|2014-08-01 00:15:00|40.7279|-73.9542|B02512|         1|        6|       1406852100|
|2014-08-01 00:17:00| 40.721|-73.9937|B02512|         1|        6|       1406852220|
|2014-08-01 00:19:00|40.7195| -74.006|B02512|         1|        6

In [ ]:
uber_data_2 = uber_data.drop("DateTimeName")
uber_data_2.show()

+-------+--------+------+----------+---------+-----------------+
|    Lat|     Lon|  Base|dayofmonth|dayofweek|DateTimeAsInteger|
+-------+--------+------+----------+---------+-----------------+
|40.7366|-73.9906|B02512|         1|        6|       1406851380|
| 40.726|-73.9918|B02512|         1|        6|       1406851740|
|40.7209|-74.0507|B02512|         1|        6|       1406851920|
|40.7387|-73.9856|B02512|         1|        6|       1406851920|
|40.7323|-74.0077|B02512|         1|        6|       1406851920|
|40.7349|-74.0033|B02512|         1|        6|       1406851980|
|40.7279|-73.9542|B02512|         1|        6|       1406852100|
| 40.721|-73.9937|B02512|         1|        6|       1406852220|
|40.7195| -74.006|B02512|         1|        6|       1406852340|
|40.7448|-73.9799|B02512|         1|        6|       1406852400|
|40.7399|-74.0057|B02512|         1|        6|       1406852460|
|40.7651|-73.9683|B02512|         1|        6|       1406852700|
|40.7354|-74.0081|B02512|

In [14]:
from handyspark import *
uber_plot = uber_data.toHandy()
uber_plot.show()

/content/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/content/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+-------------------+-------+--------+------+----------+---------+-----------------+
|       DateTimeName|    Lat|     Lon|  Base|dayofmonth|dayofweek|DateTimeAsInteger|
+-------------------+-------+--------+------+----------+---------+-----------------+
|2014-08-01 00:03:00|40.7366|-73.9906|B02512|         1|        6|       1406851380|
|2014-08-01 00:09:00| 40.726|-73.9918|B02512|         1|        6|       1406851740|
|2014-08-01 00:12:00|40.7209|-74.0507|B02512|         1|        6|       1406851920|
|2014-08-01 00:12:00|40.7387|-73.9856|B02512|         1|        6|       1406851920|
|2014-08-01 00:12:00|40.7323|-74.0077|B02512|         1|        6|       1406851920|
|2014-08-01 00:13:00|40.7349|-74.0033|B02512|         1|        6|       1406851980|
|2014-08-01 00:15:00|40.7279|-73.9542|B02512|         1|        6|       1406852100|
|2014-08-01 00:17:00| 40.721|-73.9937|B02512|         1|        6|       1406852220|
|2014-08-01 00:19:00|40.7195| -74.006|B02512|         1|        6

In [ ]:
uber_plot.printSchema()

root
 |-- DateTimeName: timestamp (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Lon: double (nullable = true)
 |-- Base: string (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- DateTimeAsInteger: long (nullable = true)



In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 4, figsize=(20, 4))
uber_plot.cols['Date/Time'].hist(ax=axs[0])
uber_plot.cols['Lat'].hist(ax=axs[1])
uber_plot.cols['Lon'].hist(ax=axs[2])
uber_plot.cols['Base'].hist(ax=axs[2])

In [15]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=["Lat", "Lon","dayofmonth","dayofweek", "DateTimeAsInteger"], outputCol="features")
new_uber = vecAssembler.transform(uber_plot)
new_uber.show()

+-------------------+-------+--------+------+----------+---------+-----------------+--------------------+
|       DateTimeName|    Lat|     Lon|  Base|dayofmonth|dayofweek|DateTimeAsInteger|            features|
+-------------------+-------+--------+------+----------+---------+-----------------+--------------------+
|2014-08-01 00:03:00|40.7366|-73.9906|B02512|         1|        6|       1406851380|[40.7366,-73.9906...|
|2014-08-01 00:09:00| 40.726|-73.9918|B02512|         1|        6|       1406851740|[40.726,-73.9918,...|
|2014-08-01 00:12:00|40.7209|-74.0507|B02512|         1|        6|       1406851920|[40.7209,-74.0507...|
|2014-08-01 00:12:00|40.7387|-73.9856|B02512|         1|        6|       1406851920|[40.7387,-73.9856...|
|2014-08-01 00:12:00|40.7323|-74.0077|B02512|         1|        6|       1406851920|[40.7323,-74.0077...|
|2014-08-01 00:13:00|40.7349|-74.0033|B02512|         1|        6|       1406851980|[40.7349,-74.0033...|
|2014-08-01 00:15:00|40.7279|-73.9542|B02512| 

In [16]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import Row
from pyspark.sql.types import FloatType

rows = []
evaluator = ClusteringEvaluator()

for k in range(2, 11):
    kmeans = KMeans(k=k, seed=1, maxIter=10)
    model = kmeans.fit(new_uber.select('features'))
    transformed = model.transform(new_uber)
    trainingCost = model.summary.trainingCost
    clusterSizes = model.summary.clusterSizes
    silhouette_scores = evaluator.evaluate(transformed)
    row = [k,trainingCost,str(clusterSizes),silhouette_scores]
    rows += [row]

result = ["K", "trainingCost", "clusterSizes","silhouette"]
cluster = spark.createDataFrame(rows, result)

cluster.show()


+---+--------------------+--------------------+------------------+
|  K|        trainingCost|        clusterSizes|        silhouette|
+---+--------------------+--------------------+------------------+
|  2|1.231970377739506...|    [402219, 427056]|0.7928968861525764|
|  3|5.505406554454890...|[277980, 266160, ...|0.7543591269576767|
|  4|3.035821592527697...|[240669, 216744, ...|0.7501073661421657|
|  5|2.318195668634442...|[231982, 143882, ...|0.7046603590613434|
|  6|1.605098063854405...|[158234, 142077, ...|0.7010457555912631|
|  7|1.022248939065816E16|[122545, 135498, ...| 0.713975730873473|
|  8| 7.70988877082168E15|[92718, 109686, 8...| 0.711629258670217|
|  9|5.989894551915797E15|[84628, 84784, 10...|0.7146278653610421|
| 10|4.978943128704091E15|[95465, 80424, 75...|0.7037236482017154|
+---+--------------------+--------------------+------------------+



In [17]:
train, test = new_uber.randomSplit([0.8, 0.2], seed=1)
print('Count of Training Dataset : ' , train.count())
print('Count of Testing Dataset: ' , test.count())

Count of Training Dataset :  663303
Count of Testing Dataset:  165972


In [18]:
#we choose k = 5
k_means = KMeans(k = 5, seed = 1, maxIter = 10, featuresCol = "features", predictionCol = "cluster")

k_model = kmeans.fit(train)
k_transformed = k_model.transform(test)

k_transformed.show()


+-------------------+-------+--------+------+----------+---------+-----------------+--------------------+----------+
|       DateTimeName|    Lat|     Lon|  Base|dayofmonth|dayofweek|DateTimeAsInteger|            features|prediction|
+-------------------+-------+--------+------+----------+---------+-----------------+--------------------+----------+
|2014-08-01 00:00:00|40.6994|-73.9591|B02617|         1|        6|       1406851200|[40.6994,-73.9591...|         1|
|2014-08-01 00:00:00|40.7623|-73.9751|B02617|         1|        6|       1406851200|[40.7623,-73.9751...|         1|
|2014-08-01 00:01:00|40.6447| -73.782|B02617|         1|        6|       1406851260|[40.6447,-73.782,...|         1|
|2014-08-01 00:01:00|40.6906|-73.9933|B02598|         1|        6|       1406851260|[40.6906,-73.9933...|         1|
|2014-08-01 00:01:00|40.7073|-74.0054|B02598|         1|        6|       1406851260|[40.7073,-74.0054...|         1|
|2014-08-01 00:02:00|40.7199|-74.0055|B02598|         1|        

In [41]:
k_transformed.printSchema()

root
 |-- DateTimeName: timestamp (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Lon: double (nullable = true)
 |-- Base: string (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- DateTimeAsInteger: long (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: integer (nullable = false)



In [70]:
from cassandra.cluster import Cluster
cluster=Cluster()
session=cluster.connect()

session.execute("create keyspace datasetkeyspace with replication={'class': 'SimpleStrategy', 'replication_factor': 1};")

session = cluster.connect('datasetkeyspace');
 
q = "CREATE TABLE uberdataset (\
id int, DateTimeName timestamp, Lat double, Lon double, Base text, \
dayofmonth int, dayofweek int, DateTimeAsInteger int, cluster int, \
PRIMARY KEY (cluster, Lat, Lon, id))"

session.execute(q)


In [ ]:
from cassandra.query import SimpleStatement, BatchStatement
batch=BatchStatement()
batch2=BatchStatement()
batch3=BatchStatement()
batch4=BatchStatement()

k_transformed1, k_transformed2, k_transformed3, k_transformed4 = k_transformed.randomSplit([0.2, 0.2, 0.2, 0.2], seed=1)

count=1
for row in k_transformed1.collect():
    batch.add(SimpleStatement("INSERT INTO uberdataset (id, DateTimeName, Lat, Lon, Base, dayofmonth, dayofweek, DateTimeAsInteger, cluster\
     VALUES({count}, {row.DateTimeName}, {row.Lat}, {row.Lon}, {row.Base}, {row.dayofmonth}, {row.dayofweek}, {row.DateTimeAsInteger} ,{row.prediction})"))
    count = count + 1
session.execute(batch)

count2 = count
for row2 in k_transformed2.collect():
    batch2.add(SimpleStatement("INSERT INTO uberdataset (id, DateTimeName, Lat, Lon, Base, dayofmonth, dayofweek, DateTimeAsInteger, cluster) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"),\
             (count2, row2.DateTimeName, row2.Lat, row2.Lon, row2.Base, row2.dayofmonth, row2.dayofweek , row2.DateTimeAsInteger, row2.prediction))
    count2 = count2 + 1
session.execute(batch2)

count3 = count2

for row3 in k_transformed3.collect():
    batch3.add(SimpleStatement("INSERT INTO uberdataset (id, DateTimeName, Lat, Lon, Base, dayofmonth, dayofweek, DateTimeAsInteger, cluster) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"),\
             (count3, row3.DateTimeName, row3.Lat, row3.Lon, row3.Base, row3.dayofmonth, row3.dayofweek, row3.DateTimeAsInteger, row3.prediction))
    count3 = count3 + 1
session.execute(batch3)
count4 = count3

for row4 in k_transformed4.collect():
    batch4.add(SimpleStatement("INSERT INTO uberdataset (id, DateTimeName, Lat, Lon, Base, dayofmonth, dayofweek, DateTimeAsInteger, cluster) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"),\
             (count4, row4.DateTimeName, row4.Lat, row4.Lon, row4.Base, row4.dayofmonth, row4.dayofweek, row4.DateTimeAsInteger, row4.prediction))
    count4 = count4 + 1
session.execute(batch4)

In [71]:
k_transformed1, k_transformed2, k_transformed3 = k_transformed.randomSplit([0.3, 0.3, 0.4], seed=1)

In [85]:
#store data in cassandra
count=1
for row in k_transformed1.collect():  
    q1 = f"INSERT INTO uberdataset(id, DateTimeName, Lat, Lon, Base, dayofmonth, dayofweek, DateTimeAsInteger, cluster) \
    VALUES ({count}, '{row.DateTimeName}', {row.Lat}, {row.Lon}, '{row.Base}', {row.dayofmonth}, {row.dayofweek}, {row.DateTimeAsInteger} ,{row.prediction})"
    session.execute(q1)
    count = count + 1 

In [87]:
#Question 1
result = session.execute("SELECT cluster, COUNT (id) AS IDs \
 FROM uberdataset GROUP BY cluster ALLOW FILTERING")
for row in result:
    print(row)

Row(cluster=5, ids=5050)
Row(cluster=1, ids=5612)
Row(cluster=8, ids=3999)
Row(cluster=0, ids=5474)
Row(cluster=2, ids=5671)
Row(cluster=4, ids=6421)
Row(cluster=7, ids=3899)
Row(cluster=6, ids=5142)
Row(cluster=9, ids=4640)
Row(cluster=3, ids=3813)


In [116]:
#Question 2
import datetime as DT

result = session.execute("SELECT id, DateTimeName, Lat, Lon, Base, dayofmonth, dayofweek, DateTimeAsInteger, cluster FROM uberdataset ALLOW FILTERING")
#lastDate = result.next(1)
last = result.all
for row in result:
    firstDate = row

week_current = firstDate.datetimename
week_ago = week_current - DT.timedelta(days=7)
print(firstDate.datetimename)
print(week_ago)
q = f"SELECT id, DateTimeName, Lat, Lon, Base, dayofmonth, dayofweek, DateTimeAsInteger, cluster FROM uberdataset\
        WHERE DateTimeName >= ('{week_ago}') and DateTimeName <= ('{week_current}') ALLOW FILTERING"

q2 = session.execute(q)
for row2 in q2:
    print(row2)


2014-08-31 17:54:00
2014-08-24 17:54:00
Row(id=43143, datetimename=datetime.datetime(2014, 8, 26, 6, 53), lat=40.5547, lon=-74.539, base='B02682', dayofmonth=26, dayofweek=3, datetimeasinteger=1409035980, cluster=8)
Row(id=41809, datetimename=datetime.datetime(2014, 8, 24, 18, 40), lat=40.5689, lon=-73.8618, base='B02617', dayofmonth=24, dayofweek=1, datetimeasinteger=1408905600, cluster=8)
Row(id=43462, datetimename=datetime.datetime(2014, 8, 26, 12, 37), lat=40.5799, lon=-73.9854, base='B02682', dayofmonth=26, dayofweek=3, datetimeasinteger=1409056620, cluster=8)
Row(id=43507, datetimename=datetime.datetime(2014, 8, 26, 13, 41), lat=40.5833, lon=-73.9611, base='B02764', dayofmonth=26, dayofweek=3, datetimeasinteger=1409060460, cluster=8)
Row(id=24699, datetimename=datetime.datetime(2014, 8, 25, 13, 8), lat=40.5864, lon=-73.9296, base='B02598', dayofmonth=25, dayofweek=2, datetimeasinteger=1408972080, cluster=8)
Row(id=24414, datetimename=datetime.datetime(2014, 8, 24, 18, 5), lat=40.

Streaming output truncated to the last 5000 lines.
Row(id=26398, datetimename=datetime.datetime(2014, 8, 29, 12, 42), lat=40.7453, lon=-73.9905, base='B02598', dayofmonth=29, dayofweek=6, datetimeasinteger=1409316120, cluster=7)
Row(id=45622, datetimename=datetime.datetime(2014, 8, 28, 11, 7), lat=40.7453, lon=-73.986, base='B02682', dayofmonth=28, dayofweek=5, datetimeasinteger=1409224020, cluster=7)
Row(id=25932, datetimename=datetime.datetime(2014, 8, 28, 11, 43), lat=40.7453, lon=-73.9823, base='B02598', dayofmonth=28, dayofweek=5, datetimeasinteger=1409226180, cluster=7)
Row(id=45731, datetimename=datetime.datetime(2014, 8, 28, 13, 23), lat=40.7454, lon=-74.0023, base='B02617', dayofmonth=28, dayofweek=5, datetimeasinteger=1409232180, cluster=7)
Row(id=25464, datetimename=datetime.datetime(2014, 8, 27, 12, 11), lat=40.7454, lon=-73.9893, base='B02598', dayofmonth=27, dayofweek=4, datetimeasinteger=1409141460, cluster=7)
Row(id=26063, datetimename=datetime.datetime(2014, 8, 28, 17,

In [119]:
#Question 3

print(firstDate.datetimename)
print(week_ago)

q3 = f"SELECT cluster, COUNT (id) AS IDs FROM uberdataset\
        WHERE DateTimeName >= ('{week_ago}') and DateTimeName <= ('{week_current}') GROUP BY cluster ALLOW FILTERING"

q_3 = session.execute(q3)
for row in q_3:
    print(row)


2014-08-31 17:54:00
2014-08-24 17:54:00
Row(cluster=8, ids=3319)
Row(cluster=7, ids=3899)
Row(cluster=3, ids=3299)


In [149]:
#Question 4
import datetime as DT

day_current = firstDate.datetimename
tendays_ago = day_current - DT.timedelta(days=10)

#print(day_current,"----",tendays_ago)

result4 = session.execute(f"SELECT cluster, Lat, Lon, id FROM uberdataset\
   WHERE DateTimeName >= ('{tendays_ago}') and DateTimeName <= ('{day_current}') \
   GROUP BY cluster, Lat, Lon, id ALLOW FILTERING")
for row in result4:
    print(row)

Row(cluster=8, lat=40.5547, lon=-74.539, id=43143)
Row(cluster=8, lat=40.5689, lon=-73.8618, id=41809)
Row(cluster=8, lat=40.5799, lon=-73.9854, id=43462)
Row(cluster=8, lat=40.5833, lon=-73.9611, id=43507)
Row(cluster=8, lat=40.5837, lon=-73.9476, id=41664)
Row(cluster=8, lat=40.5864, lon=-73.9296, id=24699)
Row(cluster=8, lat=40.5882, lon=-73.665, id=24414)
Row(cluster=8, lat=40.5897, lon=-73.9642, id=42474)
Row(cluster=8, lat=40.5915, lon=-73.9926, id=43401)
Row(cluster=8, lat=40.5917, lon=-73.7889, id=24428)
Row(cluster=8, lat=40.5945, lon=-73.7893, id=41319)
Row(cluster=8, lat=40.6014, lon=-73.9883, id=24949)
Row(cluster=8, lat=40.602, lon=-73.9941, id=43047)
Row(cluster=8, lat=40.6029, lon=-73.9639, id=42658)
Row(cluster=8, lat=40.6055, lon=-73.9497, id=44133)
Row(cluster=8, lat=40.6066, lon=-74.0035, id=25016)
Row(cluster=8, lat=40.6092, lon=-73.9532, id=24385)
Row(cluster=8, lat=40.6106, lon=-74.0245, id=42612)
Row(cluster=8, lat=40.612, lon=-74.024, id=42669)
Row(cluster=8, la

Streaming output truncated to the last 5000 lines.
Row(cluster=0, lat=40.711, lon=-73.9496, id=38414)
Row(cluster=0, lat=40.711, lon=-73.9455, id=41066)
Row(cluster=0, lat=40.7111, lon=-74.0058, id=23534)
Row(cluster=0, lat=40.7111, lon=-74.0054, id=38612)
Row(cluster=0, lat=40.7111, lon=-73.9605, id=40243)
Row(cluster=0, lat=40.7111, lon=-73.9598, id=40916)
Row(cluster=0, lat=40.7111, lon=-73.9585, id=40961)
Row(cluster=0, lat=40.7111, lon=-73.9505, id=24026)
Row(cluster=0, lat=40.7111, lon=-73.9498, id=39907)
Row(cluster=0, lat=40.7112, lon=-74.016, id=40995)
Row(cluster=0, lat=40.7112, lon=-74.0059, id=38822)
Row(cluster=0, lat=40.7112, lon=-73.9659, id=41210)
Row(cluster=0, lat=40.7112, lon=-73.9635, id=38630)
Row(cluster=0, lat=40.7112, lon=-73.9609, id=24176)
Row(cluster=0, lat=40.7112, lon=-73.9412, id=23949)
Row(cluster=0, lat=40.7114, lon=-74.0055, id=40657)
Row(cluster=0, lat=40.7114, lon=-73.9673, id=40170)
Row(cluster=0, lat=40.7114, lon=-73.9616, id=40067)
Row(cluster=0, l


Row(cluster=7, lat=40.7299, lon=-73.9915, id=44999)
Row(cluster=7, lat=40.7299, lon=-73.9915, id=45017)
Row(cluster=7, lat=40.7299, lon=-73.9867, id=44749)
Row(cluster=7, lat=40.7299, lon=-73.9853, id=45448)
Row(cluster=7, lat=40.7299, lon=-73.9808, id=44443)
Row(cluster=7, lat=40.7299, lon=-73.9747, id=44307)
Row(cluster=7, lat=40.7299, lon=-73.9583, id=46469)
Row(cluster=7, lat=40.73, lon=-74.0083, id=45624)
Row(cluster=7, lat=40.73, lon=-73.9912, id=45608)
Row(cluster=7, lat=40.7301, lon=-74.0072, id=46780)
Row(cluster=7, lat=40.7301, lon=-74.0034, id=44519)
Row(cluster=7, lat=40.7301, lon=-73.9937, id=25689)
Row(cluster=7, lat=40.7301, lon=-73.9836, id=46578)
Row(cluster=7, lat=40.7301, lon=-73.9557, id=45453)
Row(cluster=7, lat=40.7302, lon=-74.0048, id=46406)
Row(cluster=7, lat=40.7302, lon=-73.9914, id=45361)
Row(cluster=7, lat=40.7302, lon=-73.9894, id=45252)
Row(cluster=7, lat=40.7302, lon=-73.9865, id=46627)
Row(cluster=7, lat=40.7302, lon=-73.9861, id=44279)
Row(cluster=7, 


Row(cluster=3, lat=40.7601, lon=-73.831, id=48284)
Row(cluster=3, lat=40.7602, lon=-73.9949, id=26694)
Row(cluster=3, lat=40.7602, lon=-73.9794, id=27225)
Row(cluster=3, lat=40.7602, lon=-73.9757, id=47095)
Row(cluster=3, lat=40.7602, lon=-73.9669, id=47069)
Row(cluster=3, lat=40.7602, lon=-73.9278, id=47226)
Row(cluster=3, lat=40.7602, lon=-73.9215, id=47007)
Row(cluster=3, lat=40.7603, lon=-73.999, id=48167)
Row(cluster=3, lat=40.7603, lon=-73.97, id=26923)
Row(cluster=3, lat=40.7604, lon=-73.9992, id=48511)
Row(cluster=3, lat=40.7604, lon=-73.9708, id=47012)
Row(cluster=3, lat=40.7604, lon=-73.9677, id=46982)
Row(cluster=3, lat=40.7604, lon=-73.9602, id=47433)
Row(cluster=3, lat=40.7604, lon=-73.909, id=27103)
Row(cluster=3, lat=40.7605, lon=-73.9912, id=47607)
Row(cluster=3, lat=40.7605, lon=-73.9897, id=48014)
Row(cluster=3, lat=40.7605, lon=-73.9895, id=26912)
Row(cluster=3, lat=40.7606, lon=-73.99, id=47495)
Row(cluster=3, lat=40.7606, lon=-73.9898, id=47521)
Row(cluster=3, lat